In [219]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 17
year = 2022

data = get_data(day=day, year=year)

# data
# data = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"

In [ ]:
rocks = """####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##""".split("\n\n")

rocks = [set((x,y) 
           for y,l in enumerate(reversed(r.split("\n"))) for x,c in enumerate(l) if c == '#')
         for r in rocks]

rocks[2]


In [280]:
width = 7

move = lambda r,d: {(x+d[0],y+d[1]) for x,y in r}
jets = {'<': (-1,0), '>': (1,0)}

height = lambda s: max(y for _,y in s)
is_valid = lambda rock,cave: all(0 <= x < width for x,_ in rock) and (len(rock & cave) == 0)


def print_cave(cave, rock={}):
    for y in reversed(range(0, height(cave | rock)+1)):
        print('|', end='')
        for x in range(width):
            print('#' if (x,y) in cave else ('@' if (x,y) in rock else '.'), end='')
        print('|')
    print("")


def play_game(tot_rocks,
              cycle_height_start=False, cycle_height_len=False):
    cave = set([(x,0) for x in range(width)])

    true_cycle_start = False
    skipped_cycles = 0
    cycle_rocks_len = 0
    
    j = 0
    for num_rocks in range(tot_rocks):
        h = height(cave)
        
        # Skip cycles if any:
        if cycle_height_start:
            if num_rocks == tot_rocks - skipped_cycles*cycle_rocks_len:
                print(f"Reached {num_rocks} rocks + {skipped_rocks} skipped -> ",
                    f"{num_rocks + skipped_rocks}")
                
                real_height = h + skipped_cycles*cycle_height_len
                print(f"Height: {h} + {skipped_cycles*cycle_height_len} = ",
                    f"{real_height}")
                return cave, real_height

            if not true_cycle_start:
                if (h > cycle_height_start):
                    print(f"Cycle starts at: {num_rocks} rocks | height: {h}")
                    true_cycle_start = h
                    cycle_rocks_start = num_rocks
            elif not skipped_cycles and (h-true_cycle_start == cycle_height_len):
                print(f"Full cycle at: {num_rocks} rocks | height: {h}")
                cycle_rocks_len = num_rocks-cycle_rocks_start
                skipped_cycles = (tot_rocks-num_rocks) // cycle_rocks_len
                skipped_rocks = skipped_cycles*cycle_rocks_len
                        
                print(f"Cycle length in rocks: {cycle_rocks_len}")
                print(f"Skipping {skipped_cycles} cycles: ",
                    f"{skipped_rocks} rocks | {skipped_cycles*cycle_height_len} height")

        
        rock = move(rocks[num_rocks % len(rocks)], (2, h+4))

        while rock:
            jet = jets[data[j % len(data)]]
            j += 1
            new_pos = move(rock, jet)
            if is_valid(new_pos, cave):
                rock = new_pos
            new_pos = move(rock, (0, -1))
            if is_valid(new_pos, cave):
                rock = new_pos
            else:
                cave |= rock
                rock = set()
                
    return cave, height(cave)


In [283]:
cave, part1 = play_game(2022)
print("Part 1:", part1)

15942


In [289]:
# cave, h = play_game(10000)

marker_x_pos = range(1, 7) # (0, 7) works with real data but not test
markers = sorted({y for _,y in cave if all((x,y) in cave for x in marker_x_pos)})
print(f"Found {len(markers)} markers")

cycle_height_start = False
for i,y2 in enumerate(markers):
    for y1 in markers[i+1:]:
        # if y2-y1 < 10:
        #     continue
        y3 = 2 * y2 - y1
        if y3 not in markers:
            continue
        block1 = {(x,y-y2) for x,y in cave if y2 <= y < y1}
        block2 = {(x,y-y3) for x,y in cave if y3 <= y < y2}
        if block1 == block2:
            print(y1, y2, y3)
            assert y1-y2 == y2-y3
            cycle_height_len = y1-y2
            cycle_height_start = y3
            print("Cycle start:", cycle_height_start)
            print("Cycle len:", cycle_height_len)
            break
    if cycle_height_start:
        break
    
assert cycle_height_start

_, h = play_game(1000000000000, 
                      cycle_height_start=cycle_height_start,
                      cycle_height_len=cycle_height_len)

print("\nPart 2:", h)

Found 260 markers
5961 3178 395
Cycle start: 395
Cycle len: 2783
Cycle starts at: 242 rocks | height: 398
Full cycle at: 1987 rocks | height: 3181
Cycle length in rocks: 1745
Skipping 573065901 cycles:  999999997245 rocks | 1594842402483 height
Reached 2755 rocks + 999999997245 skipped ->  1000000000000
Height: 4399 + 1594842402483 =  1594842406882

Part 2: 1594842406882


104
6043 3260 477
Cycle start: 477
Cycle len: 2783


In [215]:
# 1514285714288 - (2024 + 680272106*2226 + 4318)

1514285714288 - 1514285714288

0

In [211]:
# 1000000000000 
(1000000000000-37) // 35, (1000000000000-37) % 35

(28571428570, 13)

In [ ]:
Cycle starts: 37 bricks | height: 63
full cycle: 72 tot bricks | tot height: 116
cycle brick length: 35


In [ ]:
# min_flows = dict()
# # nx.bfs_tree(G, 'AA', depth_limit=30)
#     # print(t)
# cur_states = {('AA', (), 0)}
# tot_minutes = 30

# for minute in range(1, tot_minutes+1):
#     next_states = set()
#     for cur_node, open_valves, flow in cur_states:
#         flow_rate = sum(flows[n] for n in open_valves)
#         min_flow = flow + flow_rate * (tot_minutes-minute)
#         # print(f"Exploring: {cur_node} {open_valves} {flow}")
#         if min_flows.get(cur_node, -1) >= min_flow:
#             # print(f" -> Pruning: {flow} + {tot_minutes-minute} * {flow_rate} -> {min_flow} <= {min_flows[cur_node]}")
#             continue
#         # print(f" Adding min_flow: {cur_node} -> {min_flow}")
#         min_flows[cur_node] = min_flow
#         flow += flow_rate
#         if (cur_node not in open_valves) and flows[cur_node]:
#             next_states.add((cur_node, tuple(set([*open_valves, cur_node])), flow))
#         for next_node in G.neighbors(cur_node):
#             next_states.add((next_node, open_valves, flow))
#     if len(next_states) == 0:
#         pprint(cur_states)
#         pprint(min_flows)
#     print(f"Minute {minute}: max flow = {max(f for _,_,f in next_states)} ({len(next_states)} active states)")
#     cur_states = next_states
    
#     # pprint(min_flows)

In [ ]:
# sum(v for k,v in flows.items() if k in ('BB', 'DD', 'HH'))

    # ('BB', 'HH') ('DD', 'JJ
# flows

In [ ]:
all_valves = {k for k,v in flows.items() if v}
min_flows = dict()
# nx.bfs_tree(G, 'AA', depth_limit=30)
    # print(t)
cur_states = {(('AA', 'AA'), (), 0, ())}
tot_minutes = 26

for minute in range(1, tot_minutes+1):
    print(f"Minute {minute}: max flow = {max([0, *min_flows.values()])} ({len(cur_states)} active states)\n")
        
    next_states = set()
    for cur_nodes, open_valves, flow, prev_nodes in  sorted(cur_states, key=lambda x: x[2], reverse=True):
        flow_rate = sum(flows[n] for n in open_valves)
        flow += flow_rate
        
        min_flow = flow + flow_rate * (tot_minutes-minute)

        if min_flows.get(cur_nodes, -1) >= min_flow:
            continue
        
        min_flows[cur_nodes] = min_flow
        
        if len(open_valves) == len(all_valves):
            continue
        
            
        next_states_n = [[], []]
        for i,cur_node in enumerate(cur_nodes):
            next_states_n[i] = [([(next_node, []) 
                                 for next_node in G.neighbors(cur_node)
                                 if next_node not in (prev_nodes)]
                                + ([(cur_node, [cur_node])] if flows[cur_node] and cur_node not in open_valves else []))
                                
            ]
        debug = {(
            tuple(sorted([cur_node1, cur_node2])), 
            tuple(sorted(set([*valve1, *valve2, *open_valves]))), 
            flow,
            (cur_nodes)
        )
                            for cur_node1, valve1 in next_states_n[0]
                            for cur_node2, valve2 in next_states_n[1]
                            }

        next_states.update(debug)
        
    cur_states = next_states

    if len(cur_states) == 0:
        print(f"No more moves.")
        break
    
print(f" Max flow: {max(min_flows.values())}")
    # pprint(min_flows)
    # print(max(min_flows.values()), '\n')
    # 2532
    # 2349
    # 2715
    
    # ('BB', 'HH') ('DD', 'JJ

In [ ]:
max(min_flows.values())

In [ ]:
print(f"\nMinute {minute}: max flow = {max(f+sum(flows[n] for n in o) for _,o,f in cur_states)} ({len(cur_states)} active states)\n")

In [ ]:
p, v, f, _ = max(cur_states, key=lambda x:x[2])

In [ ]:
len(v)

In [ ]:
sum(f > 0 for f in flows.values())